# 🚀 Get started to validate the setup

This Jupyter notebook is recommended for workshop/education only.

Prerequisites:

1. Set up your computing environment
2. Install the required library in your Python environment
3. Select the correct kernel (`azureml_py310_sdkv2`) for your Jupyter notebook


## 1. Azure Open AI Test

---


In [ ]:
import os

def get_workdir():
    current_dir = os.getcwd()
    
    if current_dir.__contains__("Users"):
        base_dir = current_dir[:current_dir.find("slm-innovator-lab")]
    else:
        user = os.listdir(os.path.join(current_dir,"Users"))[0]
        base_dir = os.path.join(current_dir, "Users", user)

    work_dir = os.path.join(base_dir, "slm-innovator-lab")
    print("Base dir: ", base_dir)
    print("Work dir: ", work_dir)    
    
    return work_dir

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys

work_dir = get_workdir()
module_path = os.path.join(work_dir, "0_lab_preparation")
sys.path.append(os.path.abspath(module_path))
%store work_dir

from common import check_kernel
check_kernel()

In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv()

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not aoai_api_version:
    aoai_api_version = os.getenv("OPENAI_API_VERSION")
if not aoai_deployment_name:
    aoai_deployment_name = os.getenv("DEPLOYMENT_NAME")

try:
    client = AzureOpenAI(
        azure_endpoint = aoai_api_endpoint,
        api_key        = aoai_api_key,
        api_version    = aoai_api_version
    )
    deployment_name = aoai_deployment_name
    print("=== Initialized AzuureOpenAI client ===")
    print(f"AZURE_OPENAI_ENDPOINT={aoai_api_endpoint}")
    print(f"AZURE_OPENAI_API_VERSION={aoai_api_version}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")   
except (ValueError, TypeError) as e:
    print(e)

In [ ]:
# Create your prompt
system_message = """
You are an AI assistant that helps customers find information. As an assistant, you respond to questions in a concise and unique manner.
You can use Markdown to answer simply and concisely, and add a personal touch with appropriate emojis.

Add a witty joke starting with "By the way," at the end of your response. Do not mention the customer's name in the joke part.
The joke should be related to the specific question asked.
For example, if the question is about tents, the joke should be specifically related to tents.

Use the given context to provide a more personalized response. Write each sentence on a new line:
"""
context = """
    The Alpine Explorer Tent features a detachable partition to ensure privacy, 
    numerous mesh windows and adjustable vents for ventilation, and a waterproof design. 
    It also includes a built-in gear loft for storing outdoor essentials. 
    In short, it offers a harmonious blend of privacy, comfort, and convenience, making it a second home in nature!
"""
question = "What are features of the Alpine Explorer Tent?"

user_message = f"""
Context: {context}
Question: {question}
"""

# Simple API Call
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
  temperature=0.7,
  max_tokens=300
)

print(response.choices[0].message.content)

## (Optional) 2. Azure Document Inteligence Test

---


In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import ContentFormat

doc_intelligence_endpoint = os.getenv("AZURE_DOC_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOC_INTELLIGENCE_KEY")

try:
    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=doc_intelligence_endpoint, 
        credential=AzureKeyCredential(doc_intelligence_key),
        headers={"x-ms-useragent":"sample-code-figure-understanding/1.0.0"},
    )
    print("=== Initialized DocumentIntelligenceClient ===")
    print(f"AZURE_DOC_INTELLIGENCE_ENDPOINT={doc_intelligence_endpoint}")    
except (ValueError, TypeError) as e:
    print(e)
    
raw_data_dir = "../1_synthetic-qa-generation/raw_data"
file_path = f"{raw_data_dir}/pdf/en-imagenet-training-wrote-by-daekeun.pdf"

In [ ]:
with open(file_path, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", 
        output_content_format=ContentFormat.MARKDOWN 
    )

result = poller.result()
md_content = result.content
print(md_content)

## 3. Azure ML Test

---


In [ ]:
import os
import yaml
from datetime import datetime
snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open('../2_slm-fine-tuning-mlstudio/phi3/config.yml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_DATA_NAME']    
DATA_DIR = d['config']['DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']
IS_DEBUG = d['config']['IS_DEBUG']
USE_LOWPRIORITY_VM = d['config']['USE_LOWPRIORITY_VM']


print(f"AZURE_SUBSCRIPTION_ID={AZURE_SUBSCRIPTION_ID}")
print(f"AZURE_RESOURCE_GROUP={AZURE_RESOURCE_GROUP}")
print(f"AZURE_WORKSPACE={AZURE_WORKSPACE}")
print(f"AZURE_DATA_NAME={AZURE_DATA_NAME}")
print(f"DATA_DIR={DATA_DIR}")
print(f"CLOUD_DIR={CLOUD_DIR}")
print(f"HF_MODEL_NAME_OR_PATH={HF_MODEL_NAME_OR_PATH}")
print(f"IS_DEBUG={IS_DEBUG}")
print(f"USE_LOWPRIORITY_VM={USE_LOWPRIORITY_VM}")

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.core.exceptions import HttpResponseError

credential = DefaultAzureCredential()
ml_client = MLClient(credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE)

# from azure.identity import ClientSecretCredential
# credentials = ClientSecretCredential(
#     client_id=client_id,
#     client_secret=client_secret,
#     tenant_id=tenant_id
# )

try:
    workspace = ml_client.workspaces.get(name=AZURE_WORKSPACE)
    print(f"Connected to Azure ML Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")